# NAGuide R methods

Setup basic methods and packages used for all methods

- BiocManager could be moved to methods who are installed from BioConductor

In [1]:
packages_base_R <-
  c("BiocManager", "reshape2", "data.table", "readr", "tibble")

install_rpackage  <- function(pkg) {
  # If not installed, install the package
  if (!require(pkg, character.only = TRUE)) {
    install.packages(pkg)
    library(pkg, character.only = TRUE)
  }
  
}

# used in the large imputation function for two packages
install_bioconductor  <- function(pkg) {
  # If not installed, install the package
  if (!require(pkg, character.only = TRUE)) {
    BiocManager::install(pkg)
    library(pkg, character.only = TRUE)
  }
  
}


for (package in packages_base_R) {
  # Check if the package is already installed
  install_rpackage(pkg = package)
}


Loading required package: BiocManager



Bioconductor version '3.18' is out-of-date; the current release version '3.19'
  is available with R version '4.4'; see https://bioconductor.org/install



Loading required package: reshape2



Loading required package: data.table




Attaching package: ‘data.table’




The following objects are masked from ‘package:reshape2’:

    dcast, melt




Loading required package: readr



Loading required package: tibble



setup can be tricky... trying to integrate as much as possible into conda environment

Copied from [NAGuideR's github](https://github.com/wangshisheng/NAguideR/blob/15ec86263d5821990ad39a8d9f378cf4d76b25fb/inst/NAguideRapp/app.R#L1705-L1849) RShiny application. Adapted to run as standalone function in context of the Snakemake workflow.

- `df` and `df1` ?
- seems quite hacky
- code is only slightly adapted from repo to run here, mainly to install packages on the fly

In [2]:
nafunctions <- function(x, method = "zero") {
  df <- df1 <- as.data.frame(x)
  method <- tolower(method)
  if (method == "zero") {
    df[is.na(df)] <- 0
  }
  else if (method == "minimum") {
    df[is.na(df)] <- min(df1, na.rm = TRUE)
  }
  else if (method == "colmedian") {
    install_rpackage('e1071')
    df <- impute(df1, what = "median")
  }
  else if (method == "rowmedian") {
    install_rpackage('e1071')
    dfx <- impute(t(df1), what = "median")
    df <- t(dfx)
  }
  else if (method == "knn_impute") {
    install_bioconductor('impute')
    data_zero1 <-
      impute.knn(as.matrix(df1),
                 k = 10,
                 rowmax = 1,
                 colmax = 1)#rowmax = 0.9, colmax = 0.9
    df <- data_zero1$data
  }
  else if (method == "seqknn") {
    if (!require(SeqKnn)) {
      install.packages("src/R_NAGuideR/SeqKnn_1.0.1.tar.gz",
                       repos = NULL,
                       type = "source")
      library(SeqKnn)
    }
    df <- SeqKNN(df1, k = 10)
  }
  else if (method == "bpca") {
    install_bioconductor('pcaMethods')
    data_zero1 <-
      pcaMethods::pca(
        as.matrix(df1),
        nPcs = ncol(df1) - 1,
        method = "bpca",
        maxSteps = 100
      )
    df <- completeObs(data_zero1)
  }
  else if (method == "svdmethod") {
    install_bioconductor('pcaMethods')
    data_zero1 <-
      pcaMethods::pca(as.matrix(df1),
                      nPcs = ncol(df1) - 1,
                      method = "svdImpute")
    df <- completeObs(data_zero1)
  }
  else if (method == "lls") {
    install_bioconductor('pcaMethods')
    data_zero1 <- llsImpute(t(df1), k = 10)
    df <- t(completeObs(data_zero1))
  }
  else if (method == "mle") {
    install_rpackage('norm')
    xxm <- as.matrix(df1)
    ss <- norm::prelim.norm(xxm)
    thx <- norm::em.norm(ss)
    norm::rngseed(123)
    df <- norm::imp.norm(ss, thx, xxm)
  }
  else if (method == "qrilc") {
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('imputeLCMD')
    xxm <- t(df1)
    data_zero1 <-
      imputeLCMD::impute.QRILC(xxm, tune.sigma = 1)[[1]]
    df <- t(data_zero1)
  }
  else if (method == "mindet") {
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('imputeLCMD')
    xxm <- as.matrix(df1)
    df <- imputeLCMD::impute.MinDet(xxm, q = 0.01)
  }
  else if (method == "minprob") {
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('imputeLCMD')
    xxm <- as.matrix(df1)
    df <-
      imputeLCMD::impute.MinProb(xxm, q = 0.01, tune.sigma = 1)
  }
  else if (method == "irm") {
    install_rpackage('VIM')
    df <- irmi(df1, trace = TRUE, imp_var = FALSE)
    rownames(df) <- rownames(df1)
  }
  else if (method == "impseq") {
    install_rpackage('rrcovNA')
    df <- impSeq(df1)
  }
  else if (method == "impseqrob") {
    install_rpackage('rrcovNA')
    data_zero1 <- impSeqRob(df1, alpha = 0.9)
    df <- data_zero1$x
  }
  else if (method == "mice-norm") {
    install_rpackage('mice')
    minum <- 5
    datareadmi <- mice(df1,
                       m = minum,
                       seed = 1234,
                       method = "norm")
    newdatareadmi <- 0
    for (i in 1:minum) {
      newdatareadmi <- complete(datareadmi, action = i) + newdatareadmi
    }
    df <- newdatareadmi / minum
    rownames(df) <- rownames(df1)
  }
  else if (method == "mice-cart") {
    install_rpackage('mice')
    minum <- 5
    datareadmi <- mice(df1,
                       m = minum,
                       seed = 1234,
                       method = "cart")
    newdatareadmi <- 0
    for (i in 1:minum) {
      newdatareadmi <- complete(datareadmi, action = i) + newdatareadmi
    }
    df <- newdatareadmi / minum
    rownames(df) <- rownames(df1)
  }
  else if (method == "trknn") {
    source('src/R_NAGuideR/Imput_funcs.r')
    # sim_trKNN_wrapper <- function(data) {
    #   result <- data %>% as.matrix %>% t %>% imputeKNN(., k=10, distance='truncation', perc=0) %>% t
    #   return(result)
    # }
    # df1x <- sim_trKNN_wrapper(t(df1))
    # df<-as.data.frame(t(df1x))
    df <-
      imputeKNN(as.matrix(df),
                k = 10,
                distance = 'truncation',
                perc = 0)
    df <- as.data.frame(df)
  }
  else if (method == "rf") {
    install_rpackage("missForest")
    data_zero1 <- missForest(
      t(df1),
      maxiter = 10,
      ntree = 20 # input$rfntrees
      ,
      mtry = floor(nrow(df1) ^ (1 / 3)),
      verbose = TRUE
    )
    df <- t(data_zero1$ximp)
  }
  else if (method == "pi") {
    width <- 0.3 # input$piwidth
    downshift <- 1.8 # input$pidownshift
    for (i in 1:ncol(df1)) {
      temp <- df1[[i]]
      if (sum(is.na(temp)) > 0) {
        temp.sd <- width * sd(temp[!is.na(temp)], na.rm = TRUE)
        temp.mean <-
          mean(temp[!is.na(temp)], na.rm = TRUE) - downshift * sd(temp[!is.na(temp)], na.rm = TRUE)
        n.missing <- sum(is.na(temp))
        temp[is.na(temp)] <-
          rnorm(n.missing, mean = temp.mean, sd = temp.sd)
        df[[i]] <- temp
      }
    }
    df
  }
  # else if(method=="grr"){
  #   library(DreamAI)
  #   df<-impute.RegImpute(data=as.matrix(df1), fillmethod = "row_mean", maxiter_RegImpute = 10,conv_nrmse = 1e-03)
  # }
  else if (method == "gms") {
    # install.packages('GMSimpute')
    if (!require(GMSimpute)) {
      install.packages(
        "src/R_NAGuideR/GMSimpute_0.0.1.1.tar.gz",
        repos = NULL,
        type = "source"
      )
      
      library(GMSimpute)
    }
    
    df <- GMS.Lasso(df1,
                    nfolds = 3,
                    log.scale = FALSE,
                    TS.Lasso = TRUE)
  }
  else if (method == "msimpute") {
    install_bioconductor("msImpute")
    df <- msImpute(as.matrix(df),
                   method = 'v2')
    df <- as.data.frame(df)
  }
  else if (method == "msimpute_mnar") {
    install_bioconductor("msImpute")
    df <-
      msImpute(as.matrix(df),
               method = 'v2-mnar',
               group = rep(1, dim(df)[2]))
    df <- as.data.frame(df)
  }
  else if (method == "gsimp") {
    options(stringsAsFactors = F)
    # dependencies parly for sourced file
    
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('imputeLCMD')
    install_rpackage("magrittr")
    install_rpackage("glmnet")
    install_rpackage("abind")
    install_rpackage("foreach")
    install_rpackage("doParallel")
    source('src/R_NAGuideR/GSimp.R')
    
    # wrapper function with data pre-processing
    pre_processing_GS_wrapper <- function(data_raw_log) {
      # samples in rows, features in columns #
      # Initialization #
      data_raw_log_qrilc <- as.data.frame(data_raw_log) %>%
        impute.QRILC() %>% extract2(1)
      # Centralization and scaling #
      data_raw_log_qrilc_sc <-
        scale_recover(data_raw_log_qrilc, method = 'scale')
      # Data after centralization and scaling #
      data_raw_log_qrilc_sc_df <- data_raw_log_qrilc_sc[[1]]
      # Parameters for centralization and scaling (for scaling recovery) #
      data_raw_log_qrilc_sc_df_param <- data_raw_log_qrilc_sc[[2]]
      # NA position #
      NA_pos <- which(is.na(data_raw_log), arr.ind = T)
      # NA introduced to log-scaled-initialized data #
      data_raw_log_sc <- data_raw_log_qrilc_sc_df
      data_raw_log_sc[NA_pos] <- NA
      # Feed initialized and missing data into GSimp imputation #
      result <-
        data_raw_log_sc %>% GS_impute(
          .,
          iters_each = 50,
          iters_all = 10,
          initial = data_raw_log_qrilc_sc_df,
          lo = -Inf,
          hi = 'min',
          n_cores = 1,
          imp_model = 'glmnet_pred'
        )
      data_imp_log_sc <- result$data_imp
      # Data recovery #
      data_imp <- data_imp_log_sc %>%
        scale_recover(., method = 'recover',
                      param_df = data_raw_log_qrilc_sc_df_param) %>%
        extract2(1)
      return(data_imp)
    }
    df <- t(df) # samples in rows, feature in columns
    df <- pre_processing_GS_wrapper(df)
    df <- t(df) # features in rows, samples in columns
    
  }
  else{
    stop(paste("Unspported methods so far: ", method))
  }
  df <- as.data.frame(df)
  df
}

## Parameters

Choose one of the available methods. 
Some methods might fail for your dataset for unknown reasons
(and the error won't always be easy to understand)
```method
method = 'ZERO'
method = 'MINIMUM'
method = 'COLMEDIAN'
method = 'ROWMEDIAN'
method = 'KNN_IMPUTE'
method = 'SEQKNN'
method = 'BPCA'
method = 'SVDMETHOD'
method = 'LLS'
method = 'MLE'
mehtod = 'LLS'
method = 'QRILC'
method = 'MINDET'
method = 'MINPROB'
method = 'IRM'
method = 'IMPSEQ'
method = 'IMPSEQROB'
method = 'MICE-NORM'
method = 'MICE-CART'
method = 'RF'
method = 'PI'
method = 'GMS'
method = 'TRKNN',
method = 'MSIMPUTE'
method = 'MSIMPUTE_MNAR'
method = 'GSIMP'
```

In [3]:
train_split = 'runs/example/data/data_wide_sample_cols.csv' # test
folder_experiment = 'runs/example/'
method = 'KNN_IMPUTE'

In [4]:
# Parameters
train_split = "runs/alzheimer_study_2023_11_v2/data/data_wide_sample_cols.csv"
method = "IMPSEQROB"
folder_experiment = "runs/alzheimer_study_2023_11_v2"


## Dump predictions

In [5]:
df <-
  utils::read.csv(
    train_split,
    row.names = 1,
    header = TRUE,
    stringsAsFactors = FALSE
  )
df

,Sample_000,Sample_001,Sample_002,Sample_003,Sample_004,Sample_005,Sample_006,Sample_007,Sample_008,Sample_009,⋯,Sample_200,Sample_201,Sample_202,Sample_203,Sample_204,Sample_205,Sample_206,Sample_207,Sample_208,Sample_209
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A0A024QZX5;A0A087X1N8;P35237,15.91237,NA,16.11149,16.10697,15.60321,15.81161,15.49966,15.22075,15.98013,NA,⋯,NA,15.86264,15.65594,15.40126,NA,15.68225,15.79804,15.73949,15.47682,NA
A0A024R0T9;K7ER74;P02655,16.85194,16.87369,NA,17.03151,15.33051,18.61407,17.40917,17.68391,16.38625,16.58972,⋯,17.30955,16.61480,17.95339,18.19938,17.27911,16.88635,17.55400,NA,16.77948,17.26137
A0A024R3W6;A0A024R412;O60462;O60462-2;O60462-3;O60462-4;O60462-5;Q7LBX6;X5D2Q8,15.57047,15.51892,15.93532,15.80187,15.37522,15.62420,15.91185,15.38499,15.89447,15.37538,⋯,NA,15.93179,14.85871,15.12451,NA,14.91006,15.59966,15.46896,14.99496,15.17487
A0A024R644;A0A0A0MRU5;A0A1B0GWI2;O75503,16.48108,16.38694,16.41577,16.97862,16.67906,15.95764,16.23389,16.41793,16.27134,16.20970,⋯,15.95402,16.02959,16.49753,16.51284,16.51269,16.48232,15.93782,16.89773,16.13246,16.23505
A0A075B6H7,17.30138,NA,18.17480,15.96322,NA,18.31720,NA,17.21393,17.79403,NA,⋯,NA,16.48262,14.83467,13.56893,14.48257,NA,NA,NA,NA,NA
A0A075B6H9,20.24646,19.94110,19.25090,19.62797,20.44983,18.68569,20.34722,20.79604,18.99523,19.24667,⋯,NA,NA,18.26986,19.80062,20.18315,17.70485,18.15450,18.63631,14.90754,17.89344
A0A075B6I0,16.76446,18.78624,16.83217,17.85209,18.68185,17.86543,18.24995,20.30936,17.97843,19.41047,⋯,18.00550,18.54142,17.80748,19.89865,19.67420,17.03945,18.15179,17.95040,NA,17.74443
A0A075B6I1,17.58357,17.14388,15.67073,18.87749,17.08131,14.82369,18.51969,19.35640,18.56449,17.96310,⋯,16.41402,17.50510,14.01911,17.50574,20.25111,NA,16.50267,16.32073,NA,16.37054
A0A075B6I6,16.98778,NA,17.01183,14.18153,14.14021,17.04179,17.37560,16.99794,17.20311,16.75839,⋯,15.72543,17.06758,16.45849,16.32778,16.33372,16.41304,16.86003,16.40071,16.11868,15.77958


- `data.frame` does not allow abritary column names, but only valid column names...
- tibbles don't support rownames, and the imputation methods rely on normal `data.frame`s.
Save the header row for later use.

In [6]:
original_header <- colnames(readr::read_csv(
  train_split,
  n_max = 1,
  col_names = TRUE,
  skip = 0
))
feat_name <- original_header[1]
original_header[1:5]

Rows: 1 Columns: 211


── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (1): protein groups
dbl (180): Sample_000, Sample_002, Sample_003, Sample_004, Sample_005, Sampl...
lgl  (30): Sample_001, Sample_009, Sample_012, Sample_015, Sample_017, Sampl...



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "protein groups" "Sample_000"     "Sample_001"     "Sample_002"    
[5] "Sample_003"

Uncomment to test certain methods (only for debugging, as at least one method per package is tested using Github Actions)

In [7]:
# to_test <- c(
# 'ZERO',
# 'MINIMUM',
# 'COLMEDIAN',
# 'ROWMEDIAN',
# 'KNN_IMPUTE',
# 'SEQKNN',
# 'BPCA',
# 'SVDMETHOD',
# 'LLS',
# 'MLE',
# 'LLS',
# 'QRILC',
# 'MINDET',
# 'MINPROB',
# 'IRM',
# 'IMPSEQ',
# 'IMPSEQROB',
# 'MICE-NORM',
# 'MICE-CART',
# 'RF',
# 'PI',
# 'GMS', # fails to install on Windows
# 'TRKNN',
# 'MSIMPUTE'
# 'MSIMPUTE_MNAR'
# 'GSIMP'
# )

# for (method in to_test) {
#     print(method)
#     pred <- nafunctions(df, method)
# }

Impute and save predictions with original feature and column names

In [8]:
pred <- nafunctions(df, method)
pred <- tibble::as_tibble(cbind(rownames(pred), pred))
names(pred) <- original_header
pred

Loading required package: rrcovNA



Loading required package: rrcov



Loading required package: robustbase



Scalable Robust Estimators with High Breakdown Point (version 1.7-5)




Scalable Robust Estimators with High Breakdown Point for
Incomplete Data (version 0.5-1)




Warning message in prelim.norm(x):
“NAs introduced by coercion to integer range”


protein groups,Sample_000,Sample_001,Sample_002,Sample_003,Sample_004,Sample_005,Sample_006,Sample_007,Sample_008,⋯,Sample_200,Sample_201,Sample_202,Sample_203,Sample_204,Sample_205,Sample_206,Sample_207,Sample_208,Sample_209
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
P55058,21.80705,22.08953,21.70819,21.52758,21.79960,21.61364,22.04850,21.87977,21.17219,⋯,21.80553,22.16297,21.92430,22.21416,22.14407,22.14293,22.14260,22.36779,22.33516,21.92274
F8WAD8;Q9P0K1;Q9P0K1-2;Q9P0K1-3;Q9P0K1-4;Q9P0K1-5,20.84847,20.64712,20.54980,20.95019,20.62040,20.26415,20.04093,19.98445,20.29814,⋯,18.60363,19.54022,19.99809,20.02860,18.78015,20.06308,19.31926,19.69552,19.64555,19.50927
P01814,17.39639,17.19083,17.35873,17.06318,17.51453,17.50397,16.90862,18.86435,17.70187,⋯,15.83098,15.62958,15.55319,14.00337,16.23495,15.92578,16.72536,2466.87026,15.77786,16.16226
P98095;P98095-2,16.96599,2489.13553,16.30169,17.18140,2442.09704,16.81514,17.42867,16.40268,16.62182,⋯,17.10685,16.22934,17.39362,17.11035,17.49488,17.54640,16.25839,18.16435,17.47560,17.44899
A0A0A0MT71;Q6UX73;Q6UX73-2,20.05442,19.58164,19.11768,2468.89077,19.32901,19.78976,19.39339,18.18231,18.73073,⋯,19.54230,19.04480,18.66685,18.94653,18.93725,19.48507,19.31761,18.84786,19.31850,18.19888
A0A0G2JRQ6,20.64947,19.50729,18.58304,18.65313,19.51567,19.16735,19.18839,21.05779,19.15006,⋯,20.18811,19.60687,18.47814,18.71533,19.35767,2434.52179,2454.50832,18.25513,18.25613,18.13205
O15031,18.68290,18.71817,18.45739,18.46331,17.76222,2495.81782,18.43162,17.97116,18.06314,⋯,17.49441,18.39043,18.73071,18.94415,18.31661,18.07311,17.73152,18.88865,18.47595,18.18473
O95967,17.89665,17.91853,17.61380,18.19577,17.57060,17.99087,18.08149,17.83881,17.82902,⋯,17.25470,18.09496,17.81155,17.97862,17.83986,17.82417,17.92647,18.15150,18.02988,17.66907
P05546,18.89583,18.85745,18.17469,18.90033,18.64590,19.03287,18.68525,18.99569,18.42259,⋯,18.73985,18.20652,18.39032,19.35365,19.07693,18.52529,18.75880,19.06191,19.29014,19.23596


In [9]:
pred <- reshape2::melt(pred, id.vars = feat_name)
names(pred) <- c(feat_name, 'Sample ID', method)
pred <- pred[reshape2::melt(is.na(df))['value'] == TRUE, ]
pred

,protein groups,Sample ID,IMPSEQROB
,<chr>,<fct>,<dbl>
11,P13671,Sample_000,18.25068
14,A0A0A0MTQ4;Q96B86;Q96B86-3;Q96B86-4,Sample_000,18.25099
15,P02787,Sample_000,26.75364
16,P14625,Sample_000,17.81561
19,A0A0B4J231;B9A064,Sample_000,21.13848
24,P01008,Sample_000,22.21459
34,Q8IZP7,Sample_000,17.46130
39,A0A087WUT8;O95196;O95196-2,Sample_000,18.53303
40,A0A087X0S5;P12109,Sample_000,20.59724


In [10]:
dim(pred)

[1] 71601     3

In [11]:
fname = file.path(folder_experiment,
                  'preds',
                  paste0('pred_all_', toupper(method), '.csv'))
fname

[1] "runs/alzheimer_study_2023_11_v2/preds/pred_all_IMPSEQROB.csv"

In [12]:
write_csv(pred, path = fname)

Warning message:
“The `path` argument of `write_csv()` is deprecated as of readr 1.4.0.
ℹ Please use the `file` argument instead.”
